In [1]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from numpy.polynomial.polynomial import Polynomial

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

from sklearn.preprocessing import StandardScaler

In [2]:

#import training datadet

inter = pd.ExcelFile('training data_90_10_Github.xlsx')
comb = pd.read_excel(inter,'90pct')
#print(inter.info())
#comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
print(comb.shape[0])
print(comb.shape[1])
col_to_remove=[]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
        col_to_remove.append(col)
comb = comb.reset_index(drop=True)
X_CAL = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)
print(X_CAL.info())

y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])



test = pd.read_excel(inter,'10pct')
#print(inter.info())
#test = inter1.dropna(axis=0)
#print(comb.info())
test = test[test["tR (min)"] != "?"]
#preprocess
test = test.drop(col_to_remove,axis=1)
X_test = test.drop(['Project','Column','Compound','tR (min)'], axis=1)
print(X_test.info())

y_test = {'tR (min)': [round(x,4) for x in test['tR (min)']]}
y_test = pd.DataFrame(test, columns = ['tR (min)'])
#print(y_test)

1637
185
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1637 entries, 0 to 1636
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 1.8 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 152
Columns: 141 entries, H to fsulfone
dtypes: float64(95), int64(46)
memory usage: 169.7 KB
None


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def HPLC_COLUMN_TRANSFER_SEARCH(X_CAL, Y_CAL, ind_table, COL_A, COL_B):
    temp_A = ind_table[ind_table['Column'] == COL_A].index
    temp_B = ind_table[ind_table['Column'] == COL_B].index

    Compound_A = ind_table.loc[temp_A, 'Compound'].values
    Compound_B = ind_table.loc[temp_B, 'Compound'].values
    
    X_CAL_A = X_CAL[temp_A, :]
    Y_CAL_A = Y_CAL[temp_A, :]

    X_CAL_B = X_CAL[temp_B, :]
    Y_CAL_B = Y_CAL[temp_B, :]

    UNIQUE_LIST = np.unique(np.concatenate((Compound_A, Compound_B)))


    X_SEL_A = np.empty((0, X_CAL.shape[1]))
    Y_SEL_A = np.empty((0, Y_CAL.shape[1]))
    X_SEL_B = np.empty((0, X_CAL.shape[1]))
    Y_SEL_B = np.empty((0, Y_CAL.shape[1]))
    Compound = np.empty((0,), dtype=str)
    
    for unique_item in UNIQUE_LIST:

        ind_A = np.where(Compound_A == unique_item)[0]
        ind_B = np.where(Compound_B == unique_item)[0]

        if ind_A.size == 0 or ind_B.size == 0:
            continue
        elif ind_A.size == ind_B.size:
            for k in range(len(ind_A)):
                # column 11 is the pH
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Compound = np.append(Compound, unique_item)
        elif ind_A.size < ind_B.size:
            for k in range(len(ind_A)):
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Compound = np.append(Compound, unique_item)
        elif ind_A.size > ind_B.size:
            for k in range(len(ind_B)):
                temp = X_CAL_A[ind_A, 11] - X_CAL_B[ind_B[k], 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, np.mean(X_CAL_A[ind_A[ind], :], axis=0)])
                    Y_SEL_A = np.vstack([Y_SEL_A, np.mean(Y_CAL_A[ind_A[ind], :], axis=0)])
                    X_SEL_B = np.vstack([X_SEL_B, X_CAL_B[ind_B[k], :]])
                    Y_SEL_B = np.vstack([Y_SEL_B, Y_CAL_B[ind_B[k], :]])
                    Compound = np.append(Compound, unique_item)
    
    RESULTS = {
        'X_SEL_A': X_SEL_A,
        'X_SEL_B': X_SEL_B,
        'Y_SEL_A': Y_SEL_A,
        'Y_SEL_B': Y_SEL_B,
        'Compound': Compound
    }

    #if len(Y_SEL_A) == len(Y_SEL_B):
        #plt.plot(Y_SEL_A, Y_SEL_B, 'ro')
        #plt.xlabel(f'measured Rt on {COL_A} (min)')
        #plt.ylabel(f'measured Rt on {COL_B} (min)')
        #plt.show()
    
    return RESULTS

In [4]:
import numpy as np
from scipy.stats import pearsonr
from numpy.linalg import lstsq

def COLUMN_TRANSFER_TABLE_90_10(X_CAL_90, Y_CAL_90, X_CAL_10, Y_CAL_10, ind_METRIC_90, ind_METRIC_10):
    UNIQUE_LIST = np.unique(ind_METRIC_90['Column'])

    METRIC = []
    LABEL = []

    for i in range(len(UNIQUE_LIST)):
        for k in range(i + 1, len(UNIQUE_LIST)):
            unique_list_i = UNIQUE_LIST[i]
            unique_list_k = UNIQUE_LIST[k]
            print(unique_list_i)
            print(unique_list_k)

            # Filter down to desired columns per 90% of our database
            RESULTS_90 = HPLC_COLUMN_TRANSFER_SEARCH(X_CAL_90, Y_CAL_90, ind_METRIC_90, unique_list_i, unique_list_k)

            # Filter down to desired columns per 10% of our database
            RESULTS_10 = HPLC_COLUMN_TRANSFER_SEARCH(X_CAL_10, Y_CAL_10, ind_METRIC_10, unique_list_i, unique_list_k)

            if len(RESULTS_90['Compound']) >= 2:
                num_gnum = len(RESULTS_90['Compound'])
                #print(len(RESULTS_90['Y_SEL_A']))
                #print(len(RESULTS_90['Y_SEL_B']))
                
                if len(RESULTS_90['Y_SEL_A']) == len(RESULTS_90['Y_SEL_B']):
                    corr_coef = np.corrcoef(RESULTS_90['Y_SEL_A'].flatten().tolist(), RESULTS_90['Y_SEL_B'].flatten().tolist())
                    #print(corr_coef)
                    r_squared = corr_coef[0,1] ** 2
                    print(r_squared)
                    
                    # Calculate RMSE
                    slope, intercept = np.polyfit(RESULTS_90['Y_SEL_A'].flatten().tolist(), RESULTS_90['Y_SEL_B'].flatten().tolist(), 1)
                    predicted_values = np.polyval([slope, intercept], RESULTS_10['Y_SEL_A'].flatten().tolist())
                    rmse = np.sqrt(np.sum((RESULTS_10['Y_SEL_B'].flatten().tolist() - predicted_values) ** 2) / len(RESULTS_10['Y_SEL_B']))
                    print(rmse)
                else:
                    r_squared=1
                    rmse=99

                METRIC.append([num_gnum, r_squared, rmse])
                LABEL.append([unique_list_i, unique_list_k])
                
                # ANN
                XScaler=StandardScaler()
                YScaler=StandardScaler()
                temp_x=np.vstack([X_CAL_90, RESULTS_10['X_SEL_A']])
                temp_y=np.vstack([Y_CAL_90, RESULTS_10['Y_SEL_A']])
                temp_x_test = RESULTS_10['X_SEL_B']
                temp_y = np.asarray(temp_y).astype('float32')
                temp_x = np.asarray(temp_x).astype('float32')
                temp_x_test = np.asarray(temp_x_test).astype('float32')
                temp_y_test=RESULTS_10['Y_SEL_B']
                temp_y_test = np.asarray(temp_y_test).astype('float32')
    
                X_scaled=XScaler.fit_transform(temp_x)
                y_scaled=YScaler.fit_transform(temp_y)
                X_test_scaled=XScaler.transform(temp_x_test)

                # create ANN model
                model = Sequential()
                # Defining the Input layer and FIRST hidden layer, both are same!
                model.add(Dense(units=5, input_dim=141, kernel_initializer='normal', activation='relu'))
                # Defining the Second layer of the model
                # after the first layer we don't have to specify input_dim as keras configure it automatically
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # Defining the Third layer of the model
                model.add(Dense(units=10, kernel_initializer='normal', activation='relu'))
                # The output neuron is a single fully connected node 
                # Since we will be predicting a single number
                model.add(Dense(1, kernel_initializer='normal'))
                # Compiling the model
                model.compile(loss='mean_squared_error', optimizer='adam')    
                # Fitting the ANN to the Training set
                model.fit(X_scaled, y_scaled ,batch_size = 50, epochs = 50, verbose=0)

                #X_test = X_test.astype('float32')
                # Generating Predictions on testing data
                Predictions=model.predict(X_test_scaled)
                # Scaling the predicted Price data back to original price scale
                Predictions=YScaler.inverse_transform(Predictions)
                rmse_ann = np.sqrt(mean_squared_error(temp_y_test, Predictions))
                METRIC[-1].append(rmse_ann)
                #print(METRIC)

            else:
                pass  # If condition not met, do nothing

    OUTPUT = {'METRIC': METRIC, 'LABEL': LABEL}  

    return OUTPUT

   

In [5]:
ind_table_90pct=pd.DataFrame(comb, columns = ['Project','Column','Compound'])
ind_table_10pct=pd.DataFrame(test, columns = ['Project','Column','Compound'])
X_CAL = comb.drop(['Project','Column','Compound','tR (min)'], axis=1)
Y_CAL = np.asarray(y).astype('float32')
X_CAL = np.asarray(X_CAL).astype('float32')

X_TEST = test.drop(['Project','Column','Compound','tR (min)'], axis=1)
Y_TEST = np.asarray(y_test).astype('float32')
X_TEST = np.asarray(X_TEST).astype('float32')

OUTPUT=COLUMN_TRANSFER_TABLE_90_10(X_CAL, Y_CAL, X_TEST, Y_TEST, ind_table_90pct, ind_table_10pct)


ACE AQ
ACE C18
0.9494524318898016
0.2121985919102691
1/1 [==============================] - 0s 125ms/step
ACE AQ
ACE C18 PFP
0.9595904961100827
0.13974226464724093
1/1 [==============================] - 0s 78ms/step
ACE AQ
ACE CN
0.8572711674291875
0.4703567252173878
1/1 [==============================] - 0s 62ms/step
ACE AQ
ACE Phenyl
0.9467527872984454
0.11256837426619846
1/1 [==============================] - 0s 78ms/step
ACE AQ
Ascentis F5
0.5918515117317285
0.9110834390571781
1/1 [==============================] - 0s 62ms/step
ACE AQ
BEH C18 UPLC
1/1 [==============================] - 0s 78ms/step
ACE AQ
Bonus RP
0.9208914451448991
0.4283491565851291
1/1 [==============================] - 0s 70ms/step
ACE AQ
CSH C18 UPLC
0.9592258650359855
0.3111345477477924
1/1 [==============================] - 0s 62ms/step
ACE AQ
Cortecs C18+
0.9540608984117697
0.2298358829956242
1/1 [==============================] - 0s 94ms/step
ACE AQ
Eclipse RRHD Plus C18
0.9864447475655879
0.14938904185654

1/1 [==============================] - 0s 63ms/step
ACE CN
Hypersil aQ
0.857512401243476
1.0075500660516776
1/1 [==============================] - 0s 78ms/step
ACE CN
Kinetix XB-C18
0.7600510061506977
1.2162854705625752
1/1 [==============================] - 0s 78ms/step
ACE CN
Poroshell EC-C18
0.7419909683812808
1.443275652574506
1/1 [==============================] - 0s 78ms/step
ACE CN
Poroshell SB-AQ
0.91564623360051
0.42582738549423826
1/1 [==============================] - 0s 63ms/step
ACE CN
Shield RP18
0.7961637003482569
0.9228219966881452
1/1 [==============================] - 0s 78ms/step
ACE CN
Zorbax Extend C18
0.7937848218392025
1.4328065512173125
1/1 [==============================] - 0s 94ms/step
ACE CN
Zorbax SB C8
0.8262874347699783
0.930030923246158
1/1 [==============================] - 0s 62ms/step
ACE CN
Zorbax SB-CN
0.9599177417482181
0.6225560213254986
1/1 [==============================] - 0s 62ms/step
ACE Phenyl
Ascentis F5
0.5296592860754425
0.9536107181827504

1/1 [==============================] - 0s 78ms/step
Cortecs C18+
Kinetix XB-C18
0.9774581315931131
0.5592336885389998
1/1 [==============================] - 0s 78ms/step
Cortecs C18+
Poroshell EC-C18
0.9755293077308498
0.5807160163822014
1/1 [==============================] - 0s 78ms/step
Cortecs C18+
Poroshell SB-AQ
0.9543365172724609
0.5284438723311509
1/1 [==============================] - 0s 63ms/step
Cortecs C18+
Shield RP18
0.9801637169598679
0.18403409488250336
1/1 [==============================] - 0s 62ms/step
Cortecs C18+
Zorbax Extend C18
0.9571789201230329
0.3255305241094195
1/1 [==============================] - 0s 78ms/step
Cortecs C18+
Zorbax SB C8
0.9373392959782659
0.3635800837331422
1/1 [==============================] - 0s 78ms/step
Cortecs C18+
Zorbax SB-CN
0.8355508398466167
0.4164588498597427
1/1 [==============================] - 0s 63ms/step
Eclipse RRHD Plus C18
Hypersil PFP
0.6949294107830548
1.0952521836502727
1/1 [==============================] - 0s 62ms/st

In [6]:
print(OUTPUT['METRIC'])

[[77, 0.9494524318898016, 0.2121985919102691, 0.64129305], [69, 0.9595904961100827, 0.13974226464724093, 0.49120465], [65, 0.8572711674291875, 0.4703567252173878, 1.0789412], [66, 0.9467527872984454, 0.11256837426619846, 0.78281224], [64, 0.5918515117317285, 0.9110834390571781, 2.0856895], [76, 1, 99, 0.8105622], [78, 0.9208914451448991, 0.4283491565851291, 0.50020623], [76, 0.9592258650359855, 0.3111345477477924, 0.35884383], [78, 0.9540608984117697, 0.2298358829956242, 0.29017863], [76, 0.9864447475655879, 0.14938904185654633, 0.88946855], [69, 0.7318914857281743, 0.9016848901260691, 1.8342938], [72, 0.9734883837797558, 0.1250753111528761, 0.43353894], [79, 0.963441298510556, 0.5169299107254796, 0.73429847], [78, 0.9615431468987942, 0.5438712204787347, 0.8629471], [57, 0.9532612084588103, 0.3251372524219797, 0.40141016], [74, 0.959789233835495, 0.15011548355843804, 0.34491608], [75, 0.985790302451115, 0.33134163446196624, 0.49796304], [69, 0.9723451963423465, 0.355538110953882, 0.544

In [7]:
columnA=[]
columnB=[]
samp=[]
R2=[]
rmse_LR=[]
rmse_ANN=[]

for i in range(len(OUTPUT['LABEL'])):
    columnA.append(OUTPUT['LABEL'][i][0])
    columnB.append(OUTPUT['LABEL'][i][1])
    samp.append(OUTPUT['METRIC'][i][0])
    R2.append(OUTPUT['METRIC'][i][1])
    rmse_LR.append(OUTPUT['METRIC'][i][2])
    rmse_ANN.append(OUTPUT['METRIC'][i][3])
    
columnA=pd.DataFrame(columnA,columns=['columnA'])
columnB=pd.DataFrame(columnB,columns=['columnB'])
samp=pd.DataFrame(samp,columns=['# of samp'])
R2=pd.DataFrame(R2,columns=['R2'])
rmse_LR=pd.DataFrame(rmse_LR,columns=['rmse_LR'])
rmse_ANN=pd.DataFrame(rmse_ANN,columns=['rmse_ANN'])

results = pd.concat([columnA, columnB, samp, R2, rmse_LR, rmse_ANN],axis=1)

results.to_csv('COLUMN_TRANSFER_90_10.csv')
